In [29]:
import pandas as pd
### ALL NECESSARY LIBRARIES ###
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')

# for saving variables
import pickle
import os
import tiktoken

# needed for gpt
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

%run "./utilityFunctions.ipynb"

In [37]:
# Extract the ids for test and val set
test = pd.read_csv("test.csv")
val = pd.read_csv("val.csv")
subject_ids = []

def extract_subject_ids(df, subject_ids_list):
    subject_ids_list.extend(df['subject_id'].tolist())

# Apply the function to both dataframes
extract_subject_ids(test, subject_ids)
extract_subject_ids(val, subject_ids)

print("List of subject_ids:")
print(subject_ids)

List of subject_ids:
[300, 301, 306, 308, 309, 311, 314, 323, 329, 332, 334, 337, 349, 354, 359, 361, 365, 373, 378, 384, 387, 396, 399, 405, 407, 408, 410, 411, 421, 424, 431, 432, 435, 438, 442, 450, 452, 453, 461, 462, 465, 466, 467, 469, 470, 481, 302, 307, 331, 335, 346, 367, 377, 381, 382, 388, 389, 390, 395, 403, 404, 406, 413, 417, 418, 420, 422, 436, 439, 440, 472, 476, 477, 482, 483, 484, 489, 490, 492]


In [43]:
synopsisAndSentiment = pd.read_csv("synopsisAndSentiment.csv")


In [44]:
basedlinePredictionFromSynopsis_prompt = """
Your task is to predict the PHQ8 score by analyzing the patient synopsis and sentiment provided below:

Input Information:
1. Synopsis: {Synopsis}
2. Sentiment: {Sentiment}

For your reference,the PHQ8 score ranges from 0-24 and the categories are: 
- 0-4: No or minimal depression
- 5-9: Mild depression
- 10-14: Moderate depression
- 15-19: Moderately severe depression
- 20-24: Severe depression

Output the predicted PHQ8 score in a compact JSON format on a single line without whitespaces. For example:
"Score":"value from 0-24"

"""



In [45]:
basedlinePredictionFromTranscript_template = """
Your task is to predict the PHQ8 score by analyzing the patient conversation provided below:

Input Information:
1. Conversation: {Conversation}

For your reference,the PHQ8 score ranges from 0-24 and the categories are: 
- 0-4: No or minimal depression
- 5-9: Mild depression
- 10-14: Moderate depression
- 15-19: Moderately severe depression
- 20-24: Severe depression

Output the predicted PHQ8 score in a compact JSON format on a single line without whitespaces. For example:
"Score":"value from 0-24"

"""



In [41]:
# Pricing details
PROMPT_COST_PER_1000_TOKENS = 0.005
COMPLETION_COST_PER_1000_TOKENS = 0.015

# Initialize the AzureOpenAI client with the specified endpoint and API key
client = AzureOpenAI(
    azure_endpoint="https://gptshuhaotest.openai.azure.com/",  # your Azure endpoint
    api_key="d0a5a4feefc34f71b685fc394d033b2c",  # your API key
    api_version="2024-02-01"
)

MODEL = "gpt-4o"  # change model here


# keep track of conversation
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})
    
# clear conversation  
def clear_conversation():
    global conversation_history
    conversation_history = [
    ]


# @retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(10))
def get_response(prompt):
    # Add the user's message to the conversation history.
    add_message("user", prompt)
    
    # Create a chat completion request to the OpenAI API.
    response = client.chat.completions.create(
        model=MODEL,     
        # The conversation history, including system, user, and assistant messages.
        messages=conversation_history, 
        temperature=0,  # Controls the randomness of the output. 
        max_tokens=2000,  # The maximum number of tokens to generate in the response.
        top_p=1,  # Controls the diversity via nucleus sampling; 1 means use all tokens.
        frequency_penalty=2,  # Penalizes new tokens based on their frequency in the text so far.
        presence_penalty=2,  # Penalizes new tokens based on whether they appear in the text so far.
        n=1,  # The number of completions to generate. Here, we are generating only one.
        response_format={ "type": "json_object" },
    )

    # Add the assistant's message to the conversation history.
    assistant_message = response.choices[0].message.content
    add_message("assistant", assistant_message)
    
    
    # Save usage and compute cost
    usage = response.usage
    computeCost(usage)

    # Return the assistant's message.
    return assistant_message

In [42]:
import json
import pandas as pd
import random
# List to store the generated synopses
Predictions = []

for index, row in synopsisAndSentiment.iterrows():
    if row['ParticipantID'] in subject_ids:
        # Determine the number of times to prompt based on PHQ_Binary value
        prompt_multiplier = 1

        for _ in range(prompt_multiplier):
            attempt_count = 0
            success = False
            while attempt_count < 10 and not success:
                attempt_count += 1
                prompt = basedlinePredictionFromSynopsis_prompt.format(
                    Conversation=row["Conversation"],
                )
                try:
                    # Get synthetic data response
                    response = get_response(prompt)
                    print(f"Participant ID {row['ParticipantID']}")
                    print(f"Attempt {attempt_count}: {response}") 
                    # Try to parse the response as JSON
                    json_data = json.loads(response)

                    # Check if the required keys are present in the JSON data
                    if 'Score' in json_data:
                        json_data['ParticipantID'] = row['ParticipantID']  
                        json_data['Sentiment'] = row['Sentiment']
                        json_data['Synopsis'] = row['Synopsis']
                        Predictions.append(json_data)
                        success = True
                    else:
                        print("Error: Required keys are missing from the JSON data. Retrying...")
                        clear_conversation()
                        continue  # Continue to retry since the JSON does not have the required keys

                except json.JSONDecodeError:
                    # If JSON parsing fails, retry fetching the response
                    print(f"Error: Response not loadable as JSON (Attempt {attempt_count}). Retrying...")
                    clear_conversation()  # Reset conversation if needed for next attempt

                except Exception as e:
                    # Handle any other exceptions, including BadRequestError
                    print(f"Error encountered: {e}. Skipping this prompt.")
                    clear_conversation()  # Exit the while loop and skip to the next row

            clear_conversation()
            
# Convert the list of JSON objects to a DataFrame
Predictions_df = pd.DataFrame(Predictions)


Predictions_df.to_csv('baselineScoresFromTranscript.csv', index=False)

Predictions_df

Total cost for this conversation: $0.00512
Participant ID 300
Attempt 1: {
  "Score": "0"
}
Total cost for this conversation: $0.01031
Participant ID 301
Attempt 1: {"Score":"4"}
Total cost for this conversation: $0.00626
Participant ID 302
Attempt 1: {
  "Score": "10"
}
Total cost for this conversation: $0.01157
Participant ID 306
Attempt 1: {
  "Score": "5"
}
Total cost for this conversation: $0.01689
Participant ID 307
Attempt 1: {"Score":"9"}
Total cost for this conversation: $0.00775
Participant ID 308
Attempt 1: {"Score":"20"}
Total cost for this conversation: $0.00679
Participant ID 309
Attempt 1: {"Score":"15"}
Total cost for this conversation: $0.00737
Participant ID 311
Attempt 1: {
  "Score": "20"
}
Total cost for this conversation: $0.02158
Participant ID 314
Attempt 1: {"Score":"5"}
Total cost for this conversation: $0.01041
Participant ID 323
Attempt 1: {"Score":"0"}
Total cost for this conversation: $0.00807
Participant ID 329
Attempt 1: {
  "Score": "0"
}
Total cost for

,Score,ParticipantID,Sentiment,Synopsis
0,0,300,Positive emotions are expressed regarding love...,"The patient is originally from Atlanta, Georgi..."
1,4,301,Patient exhibits mixed emotions: generally pos...,"The patient, originally from Los Angeles and c..."
2,10,302,The patient's emotions range widely: nostalgia...,The patient discusses various aspects of their...
3,5,306,Throughout the session multiple emotions were ...,"The patient, originally from Colorado and now ..."
4,9,307,The patient's emotions fluctuate between frust...,The patient discusses their transition from Mo...
...,...,...,...,...
71,4,477,The patient's responses reflect a mix of emoti...,The patient discussed various aspects of their...
72,10,484,The patient expresses mixed sentiments: apprec...,The patient discusses a range of topics includ...
73,0,489,The patient's responses convey a range of emot...,"The patient, originally from San Luis Obispo a..."
74,2,490,The overall sentiment expressed includes mixed...,The patient discusses various aspects of their...
